In [7]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from scipy.integrate import odeint, solve_ivp
from typing import Tuple

torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Print only 4 decimals
np.set_printoptions(precision=4)
torch.set_printoptions(precision=4)

PATH = "../data/"

def get_data(exp_id: str, batch: bool = True) -> pd.DataFrame:
    xls = pd.ExcelFile(PATH + f"{exp_id}_for_model.xlsx")
    df = xls.parse(0)
    df.drop(0, inplace=True)
    df['exp_id'] = exp_id
    if batch:
        df = df[df["Batch"] == 0]
    return df

def concat_data():
    return pd.concat([get_data(exp_id=exp_id, batch=True) for exp_id in ["BR01", "BR02", "BR03", "BR04", "BR05", "BR06", "BR07", "BR08", "BR09"]], ignore_index=True)

def get_training_data(df: pd.DataFrame) -> Tuple[np.array, np.array]:
    t_train = df['Time'].values
    u_train = df[['Biomass', 'Glucose']].values
    return np.float32(t_train), np.float32(u_train)

def ode_func(t, y, mu_max, Km, Y_XS):
    X = y[0]
    S = y[1]
    mu = mu_max * S / (Km + S)
    return [mu * X, -1 / Y_XS * mu * X]

##### Get training data

In [11]:
df = concat_data()
print(f'Number of training points from ALL experiments: {len(df)}')

t_train, u_train = get_training_data(df)

# Train data to tensor
ts_train = torch.tensor(t_train, requires_grad=True, device=device, dtype=torch.float32).view(-1, 1)
us_train = torch.tensor(u_train, requires_grad=True, device=device, dtype=torch.float32)

Number of training points from ALL experiments: 53


##### Physics Informed Neural Network

In [12]:
class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
        self.input = nn.Linear(1, 10)
        self.hidden = nn.Linear(10, 10)
        self.output = nn.Linear(10, 2)

        self.mu_max = nn.Parameter(torch.tensor(0.5))
        self.Km = nn.Parameter(torch.tensor(0.5))
        self.Y_XS = nn.Parameter(torch.tensor(0.5))

    def forward(self, x):
        x = torch.tanh(self.input(x))
        x = torch.tanh(self.hidden(x))
        x = self.output(x)
        return x

##### Training loop

In [13]:
model = PINN().to(device)
optimizer = torch.optim.RMSprop(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()

In [14]:
n_samples = 100

def physics_loss(model: nn.Module, t_start, t_end):
    t = torch.linspace(t_start, t_end, n_samples, device=device, requires_grad=True).reshape(-1,1)
    u = model(t).to(device)
    u_X = u[:,0].view(-1,1)
    u_S = u[:,1].view(-1,1)
    u_t_X = torch.autograd.grad(u_X, t, grad_outputs=torch.ones_like(u_X), create_graph=True)[0]
    u_t_S = torch.autograd.grad(u_S, t, grad_outputs=torch.ones_like(u_S), create_graph=True)[0]
    error_X = u_t_X - model.mu_max * u_X * u_S / (u_S + model.Km)
    error_S = u_t_S + 1 / model.Y_XS * model.mu_max * u_X * u_S / (u_S + model.Km)
    return torch.mean(error_X**2) + torch.mean(error_S**2)

In [16]:
# Plot LOSS 
import matplotlib.pyplot as plt

plt.plot(LOSS)
plt.yscale('log')
plt.show()